**Quantitative Momentum Strategy**

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import math
from scipy import stats
from statistics import mean
import datetime as dt

In [ ]:
tickers = pd.read_csv('sp_500_stocks.csv')
tickers = list(tickers['Ticker'])
tickers = tickers[:100]
columns = ['Ticker', 'Price', '1mReturns', '3mReturns', '6mReturns', '12mReturns']
df = pd.DataFrame(columns=columns)
df['Ticker'] = tickers
for i in range(len(tickers)):
    stock = yf.Ticker(ticker=tickers[i])
    data = stock.info.get('currentPrice')
    df.iloc[i, 1] = data


In [ ]:
days = [30, 90, 120, 360]

for j in range(4):
    endDate = dt.datetime.now()
    startDate = endDate - pd.DateOffset(days[j])
    df1 = yf.download(tickers=tickers, start=startDate, end=endDate)
    df1 = df1['Close']
    for i in range(len(tickers)):
        df.iloc[i, j+2] = (df1[tickers[i]][-1] - df1[tickers[i]][0]) / (df1[tickers[i]][0])

columns = ['1mPercentile', '3mPercentile', '6mPercentile', '12mPercentile', 'hqm_score', 'Number of Shares to Buy']
df1 = pd.DataFrame(columns=columns)
df = pd.concat([df, df1], axis=1)
for j in range(4):
    for i in range(len(tickers)):
        df.iloc[i,j+6] = stats.percentileofscore(df.iloc[:,j+2], df.iloc[i,j+2]) / 100

In [ ]:
for i in range(len(tickers)):
    df.iloc[i, -2]  = np.sum(df.iloc[i,6:10]) / 4
df = df.sort_values(by='hqm_score', ascending=False, ignore_index=True)
df = df.iloc[:50, :]

portfolio_value = 10000000   # float(input("Enter your portfolio value"))
equal_weight = float(portfolio_value / len(tickers))
for i in range(50):
    df.iloc[i, -1] = math.floor(equal_weight/df['Price'][i])

df